### Create network for spatial_queue simulation from OSMnx

In [36]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkt import loads
from shapely import wkt

import osmnx as ox

In [2]:
### download network

case = 'el_dorado'

# define a bounding box in San Francisco
north, south, east, west = 39.12, 38.46, -120.02, -121.06
### San Francisco: 37.79, 37.78, -122.41, -122.43
### El Dorado County: 39.12, 38.46, -120.02, -121.06

# create network from that bounding box
G = ox.graph_from_bbox(north, south, east, west, network_type="drive_service")

# convert to geopandas
ox_nodes, ox_edges = ox.graph_to_gdfs(G)
print('Download OSMnx road network, containing {} nodes, {} edges'.format(
    ox_nodes.shape[0], ox_edges.shape[0]))

Download OSMnx road network, containing 32516 nodes, 73602 edges


In [54]:
### clean network

def take_average(x):
    if isinstance(x, list):
        return np.mean([int(xi.split(' ')[0]) for xi in x])
    else:
        return int(x.split(' ')[0])

### copy and re-arrange data
raw_nodes = ox_nodes.copy().reset_index()
raw_edges = ox_edges.copy().reset_index()

raw_nodes['nid'] = np.arange(raw_nodes.shape[0])
raw_nodes['lon'] = raw_nodes['x']
raw_nodes['lat'] = raw_nodes['y']
osmid_to_nid_dict = {getattr(node, 'osmid'): getattr(node, 'nid') for node in raw_nodes.itertuples()}
raw_edges['nid_s'] = raw_edges['u'].map(osmid_to_nid_dict)
raw_edges['nid_e'] = raw_edges['v'].map(osmid_to_nid_dict)
raw_edges['length'] = raw_edges['length'].astype(float)
raw_edges['lanes'] = raw_edges['lanes'].fillna('1').apply(lambda x: take_average(x))
raw_edges['maxspeed'] = raw_edges['maxspeed'].fillna('25').apply(lambda x: take_average(x))
raw_edges['geometry'] = raw_edges['geometry'].apply(wkt.dumps)

display(raw_nodes.head(1))
display(raw_edges.head(1))

/Users/bingyu/opt/anaconda3/envs/cities/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,osmid,y,x,street_count,highway,ref,geometry,nid,lon,lat
0,86303247,38.545986,-120.743901,4,NaN,NaN,POINT (-120.74390 38.54599),0,-120.743901,38.545986


,u,v,key,osmid,name,highway,oneway,length,ref,geometry,lanes,service,maxspeed,bridge,access,tunnel,junction,width,nid_s,nid_e
0,86303247,86303249,0,10282918,Oak Trail,residential,False,140.438,NaN,LINESTRING (-120.7439008999999999 38.545985999...,1.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,0,1


In [55]:
### remove edges shorter than 20 meters
removed_edges = raw_edges[raw_edges['length']<=20].copy()
print('Removed {} edges < 20m'.format(removed_edges.shape[0]))

### group the nodes of the removed edges and replace them with a virtual node
removed_node_grp = {}
grp_id = 0
for edge in removed_edges.itertuples():
    nid_s = getattr(edge, 'nid_s')
    nid_e = getattr(edge, 'nid_e')
    try:
        nid_s_grp = removed_node_grp[nid_s]
    except KeyError:
        nid_s_grp = grp_id
    try:
        nid_e_grp = removed_node_grp[nid_e]
    except KeyError:
        nid_e_grp = grp_id
    nid_se_grp_id = min(nid_s_grp, nid_e_grp)
    removed_node_grp[nid_s] = nid_se_grp_id
    removed_node_grp[nid_e] = nid_se_grp_id
    if nid_se_grp_id == grp_id: grp_id += 1
    
removed_node_grp_df = pd.DataFrame(removed_node_grp.items(), columns=['nid', 'node_grp'])
removed_node_grp_df['node_grp'] = removed_node_grp_df['node_grp'].apply(lambda x: 'g{}'.format(x))
print(' ... when edges < 20m are removed, {} nodes are removed along, and {} virtual nodes are generated'.format(
    removed_node_grp_df['nid'].nunique(), removed_node_grp_df['node_grp'].nunique()))

Removed 5408 edges < 20m
 ... when edges < 20m are removed, 4804 nodes are removed along, and 2079 virtual nodes are generated


In [60]:
### contract edges
contracted_edges = raw_edges.copy() ### remove those with duplicated new_node_id, e.g., becoming loops
contracted_edges = pd.merge(contracted_edges, removed_node_grp_df, 
                           how='left', left_on='nid_s', right_on='nid')
contracted_edges = pd.merge(contracted_edges, removed_node_grp_df, 
                           how='left', left_on='nid_e', right_on='nid', suffixes=['_ns0', '_ne0'])
contracted_edges['node_grp_ns0'] = np.where(
    pd.isnull(contracted_edges['node_grp_ns0']), contracted_edges['nid_s'], contracted_edges['node_grp_ns0'])
contracted_edges['node_grp_ne0'] = np.where(
    pd.isnull(contracted_edges['node_grp_ne0']), contracted_edges['nid_e'], contracted_edges['node_grp_ne0'])
contracted_edges = contracted_edges[[
    'nid_s', 'nid_e', 'node_grp_ns0', 'node_grp_ne0', 'length', 'lanes', 'maxspeed', 'geometry'
]]
contracted_edges = contracted_edges.loc[contracted_edges['node_grp_ns0']!=contracted_edges['node_grp_ne0']]

### contract nodes
contracted_nodes = pd.merge(raw_nodes, removed_node_grp_df, how='left', on='nid')
contracted_nodes['node_grp'] = np.where(
    pd.isnull(contracted_nodes['node_grp']), contracted_nodes['nid'], contracted_nodes['node_grp'])
contracted_nodes = contracted_nodes.groupby('node_grp').agg({'lon': np.mean, 'lat': np.mean}).reset_index()
print('Originally there are {} nodes; now there are {} nodes'.format(
    raw_nodes.shape[0], contracted_nodes.shape[0]))
### update nodes
contracted_nodes = contracted_nodes.loc[(
    contracted_nodes['node_grp'].isin(contracted_edges['node_grp_ns0'])) | 
    (contracted_nodes['node_grp'].isin(contracted_edges['node_grp_ne0']))
]
contracted_nodes['node_id'] = np.arange(contracted_nodes.shape[0])

contracted_edges = pd.merge(contracted_edges, contracted_nodes, 
                            how='left', left_on='node_grp_ns0', right_on='node_grp')
contracted_edges = pd.merge(contracted_edges, contracted_nodes, 
                            how='left', left_on='node_grp_ne0', right_on='node_grp', 
                           suffixes=['_ns', '_ne'])
print('Some nodes and links in newly formed loops are further removed, resulting in {} nodes and {} edges'.format(
    contracted_nodes.shape[0], contracted_edges.shape[0]))

### update geometry
geometry_list = []
for edge in contracted_edges.itertuples():
    geometry = getattr(edge, 'geometry').replace('LINESTRING(','').replace(')', '').split(', ')
    geometry = [tuple(xy.split(' ')) for xy in geometry]
    lon_ns, lat_ns = getattr(edge, 'lon_ns'), getattr(edge, 'lat_ns')
    lon_ne, lat_ne = getattr(edge, 'lon_ne'), getattr(edge, 'lat_ne')
    geometry = [(lon_ns, lat_ns)] + geometry[1:-2] + [(lon_ne, lat_ne)]
    geometry_list.append('LINESTRING('+', '.join('{} {}'.format(xy[0], xy[1]) for xy in geometry)+')')
contracted_edges['geometry'] = geometry_list
contracted_edges['start_nid'] = contracted_edges['node_id_ns']
contracted_edges['end_nid'] = contracted_edges['node_id_ne']
contracted_edges['nid_s_old'] = contracted_edges['nid_s']
contracted_edges['nid_e_old'] = contracted_edges['nid_e']
contracted_edges = contracted_edges[['start_nid', 'end_nid', 'nid_s_old', 'nid_e_old',
                                  'length', 'lanes', 'maxspeed', 'geometry']]
contracted_edges = contracted_edges.loc[contracted_edges['start_nid']!=contracted_edges['end_nid']]

### add attributes (speed in mph)
contracted_edges['maxmph'] = contracted_edges['maxspeed']
contracted_edges['fft'] = contracted_edges['length']/(contracted_edges['maxmph']*1609/3600)
contracted_edges = contracted_edges.sort_values(by='fft', ascending=True).drop_duplicates(
    subset=['start_nid', 'end_nid'], keep='first')
print('Some links that connect the same pair of nodes are further removed, resulting in {} nodes and {} edges'.format(
    contracted_nodes.shape[0], contracted_edges.shape[0]))

### add link_id
contracted_edges['link_id'] = np.arange(contracted_edges.shape[0])
contracted_edges = contracted_edges.reset_index(drop=True)
contracted_nodes = contracted_nodes.reset_index(drop=True)
display(contracted_edges.tail(1))
display(contracted_nodes.tail(1))

Originally there are 32516 nodes; now there are 29791 nodes
Some nodes and links in newly formed loops are further removed, resulting in 29791 nodes and 66403 edges
Some links that connect the same pair of nodes are further removed, resulting in 29791 nodes and 64449 edges


/Users/bingyu/opt/anaconda3/envs/cities/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,start_nid,end_nid,nid_s_old,nid_e_old,length,lanes,maxspeed,geometry,maxmph,fft,link_id
64448,24672,20670,29083,24311,21699.456,1.0,25.0,"LINESTRING(-120.7431867 38.9293456, -120.74321...",25.0,1942.027137,64448


,node_grp,lon,lat,node_id
29790,g999,-120.826654,38.709932,29790


In [61]:
removed_node_grp_df.to_csv('{}_nid_grp_conversion.csv'.format(case), index=False)
contracted_nodes.to_csv('{}_nodes.csv'.format(case), index=False)
contracted_edges.to_csv('{}_links.csv'.format(case), index=False)